<a href="https://colab.research.google.com/github/kgpark88/energy-bigdata-analysis/blob/main/spark_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Colab에서 PySpark 실행하는 방법
- 참조 : https://medium.com/@TheITspace/running-pyspark-on-google-colab-2552435972b3

In [ ]:
# Colab에서 PySpark 사용하는 방법 1
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=c89016cd77cdcb172b99d493df0bbbde3092c718eb1fd11e657c38c9dd7baf33
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
# # Colab에서 PySpark 사용하는 방법 2
# !apt-get install openjdk-8-jdk-headless -qq 
# !wget -q !wget -q https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz  
# !tar -xf spark-3.3.2-bin-hadoop3.tgz
# !pip install -q findspark

# import os
# import findspark

# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

# findspark.init()
# findspark.find()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("demo")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

## ngrok를 사용하여 Spark UI 접속

In [ ]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

In [ ]:
!tar xf ngrok-v3-stable-linux-amd64.tgz

In [ ]:
#  !./ngrok authtoken <authtoken > # https://dashboard.ngrok.com/
!./ngrok authtoken 68EgMVst4xkntqLofArnK_7VuGPawLqhayeXRFfNLtH

In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!ps -ef | grep ngrok

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
"import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## 데이터 로딩
- 데이터 출처 : https://www.kaggle.com/sdolezel/black-friday

In [ ]:
df = spark.read.csv("train.csv", header=True, inferSchema=True)

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.count()

## PySpark 사용법

### 특정 컬럼 데이터 보기

In [ ]:
df.select("User_ID","Gender","Age","Occupation").show(5)

### 컬럼

In [ ]:
df.describe().show()

### 범주형 컬럼(Categorical columns)의 유일값 확인

In [ ]:
df.select("City_Category").distinct().show()

### Groupby 집계


In [ ]:
from pyspark.sql import functions as F
df.groupBy("City_Category").agg(F.sum("Purchase")).show()

### Null value 확인 및 처리

In [ ]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
df = df.fillna({'Product_Category_2':0, 'Product_Category_3':0})

In [ ]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

## 데이터 저장

In [ ]:
df.write.csv("pre_processed_data")

In [ ]:
df.rdd.getNumPartitions()

## Pandas 데이터프레임으로 저장

In [ ]:
df_pd = df.toPandas()
df_pd.to_csv("pandas_pre_processed_data.csv")